---
title: 'Лабораторна робота №9. Алгоритми на рядках'
description:
   Документ зроблено за допомогою [Quarto](https://quarto.org/)
author: "Яркоий Тимофій"
date: "05.29.2024"
lang: ukr
format:
  html:
    code-fold: true
    toc: true # меню
    toc_float: # спливаюче меню  
      collapsed: true # авто
      number_sections: true
jupyter: python3
---

### Завданя для самостійної роботи

* повторити код, наведений вище для довільних рядків;
* пояснити асимптотику наївного алгоритму пошуку підрядка;
* обчислити асимптотичну складність алгоритму стиснення рядка за допомогою z-функції.

### Повторити код

### Наївний пошук шаблонів (The Naive String Matching Algorithm)



In [35]:
def naive_match(p, t):
    assert len(p) <= len(t)  # припускаємо, що довжина шаблону не більша за довжину тексту
    occurrences = []
    for i in range(0, len(t)-len(p)+1):  # для кожного вирівнювання шаблону на текст
        match = True  # припускаємо, що є збіг, доки не доведено зворотнє
        for j in range(0, len(p)):  # для кожної позиції шаблону
            if t[i+j] != p[j]:
                match = False  # принаймні один символ не збігається
                break
        if match:
            occurrences.append(i)
    return occurrences

In [37]:
t = 'abg gba aaababg' # "text" - thing we search in
p = 'ab' # "pattern" - thing we search for
naive_match(p, t)

[0, 10, 12]

In [40]:
t[9: 9 + len(p)]

'aa'

In [42]:
naive_match('abg', 'abg gba aaababg')

[0, 12]

### Z-функція



In [45]:
def z_func(s):
    # Ініціалізуємо масив Z, в якому будемо зберігати значення Z-функції
    Z = [len(s)] + [0] * len(s)  # Встановлюємо довжину рядка s у першому елементі масиву Z, а решту заповнюємо нулями
    assert len(s) > 1  # Перевіряємо, чи довжина рядка s більше 1
    
    # Початкова порівняння s[1:] з префіксом
    for i in range(1, len(s)):
        # Порівнюємо символи s[i] та s[i-1]. Якщо вони однакові, збільшуємо значення Z[1]
        if s[i] == s[i - 1]:
            Z[1] += 1
        else:
            break  # Якщо знайдений символ не збігається з попереднім, припиняємо процес
    
    # Установлюємо значення змінних r і l
    r, l = 0, 0
    if Z[1] > 0:
        r, l = Z[1], 1
    
    # Обчислення Z-функції для кожного k від 2 до довжини рядка s
    for k in range(2, len(s)):
        assert Z[k] == 0  # Перевірка, що значення Z[k] спочатку дорівнює 0
        if k > r:
            # Випадок 1: k за межами поточного діапазону [l, r]
            for i in range(k, len(s)):
                # Порівнюємо символи s[i] та s[i-k]. Якщо вони збігаються, збільшуємо значення Z[k]
                if s[i] == s[i - k]:
                    Z[k] += 1
                else:
                    break
            # Оновлюємо значення змінних r та l
            r, l = k + Z[k] - 1, k
        else:
            # Випадок 2: k знаходиться всередині поточного діапазону [l, r]
            # Обчислюємо довжину бета (nbeta) та значення Z-функції на попередньому кроці (Zkp)
            nbeta = r - k + 1
            Zkp = Z[k - l]
            if nbeta > Zkp:
                # Підвипадок 2а: Zkp виграє
                Z[k] = Zkp
            else:
                # Підвипадок 2б: Порівнюємо символи, що знаходяться безпосередньо після r
                nmatch = 0
                for i in range(r + 1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                Z[k] = r - k + 1
    return Z

z_func('abggbaaaababg')

[13, 0, 0, 0, 0, 1, 1, 1, 2, 0, 3, 0, 0, 0]

### Пошук підрядка в рядку



In [48]:
def zMatch(p, t):
    # Формуємо розширений рядок s, де p – це шаблон, а t – текст, який шукаємо.
    s = p + "$" + t
    
    # Обчислюємо масив Z для рядка s.
    Z = z_func(s)
    
    # Ініціалізуємо список occurrences для зберігання позицій, де відбувається збіг з шаблоном.
    occurrences = []
    
    # Проходимо по всіх позиціях збігу шаблону з текстом у рядку s.
    # Починаємо з позиції, що відповідає кінцю шаблону (len(p) + 1) і до кінця рядка s.
    for i in range(len(p) + 1, len(s)):
        # Якщо значення Z на поточній позиції рівне довжині шаблону, то це означає, що відбувся збіг.
        if Z[i] == len(p):
            # Додаємо позицію, де відбувся збіг, до списку occurrences.
            # Позиція розраховується як i – (len(p) + 1), оскільки ми розглядаємо розширений рядок s,
            # де шаблон p розташований на початку, перед "$".
            occurrences.append(i - (len(p) + 1))
    
    # Повертаємо список знайдених збігів.
    return occurrences
t, p = "abg gba aaababg", "ab"
calculated_z = z_func("ab$abg gba aaababg")
print(calculated_z)


[18, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 1, 2, 0, 2, 0, 0, 0]


### Стиснення рядка


In [51]:
def compress_with_z(s):
    # Обчислюємо Z-вектор для рядка s
    z_vec = z_func(s)
    
    # Проходимо по елементах Z-вектора, починаючи з індексу 1
    for i in range(1, len(s)):
        # Перевіряємо, чи виконуються умови для оптимального стиснення
        if (i + z_vec[i] == len(s)) and (z_vec[i] % i == 0):
            # Якщо умови виконуються, повертаємо частину рядка s, яка буде стиснута
            return s[:i]
    
    # Якщо жодна пара (i, z_vec[i]) не відповідає умовам, повертаємо вихідний рядок s без змін
    else:
        return s

s = "aaababgaaababgaaababg"
print(z_func(s))
print(compress_with_z(s))


[21, 2, 1, 0, 1, 0, 0, 14, 2, 1, 0, 1, 0, 0, 7, 2, 1, 0, 1, 0, 0, 0]
aaababg


### Пояснити асимптотику наївного алгоритму пошуку підрядка

t - текст, p - зразок. 
Алгоритм порівнює 1 символ p з i символом t

Якщо вони співпадають, то порівнюється наступний символ p з наступним символом t, якщо є співпадіння, то ми знайшли підрядок, i = кінець підрядка в t, далі повертаємось на початок і знову порівнюємо i символ t з першим символом p.

Якшо співпадіння  не відбулось то i=i+1, і повертаємось на перший крок.

Через проходження по всім символам t, а також коли є співпадіння то і по всім символам p, тому складність в найгіршому випадку O(|p|\*|t|). (цикл в циклі).

### Обчислити асимптотичну складність алгоритму стиснення рядка за допомогою z-функції.

Стиснення рядка складається з двох частин. Перша це обчислення z-функції, асимптотична складність цього процесу 𝑂(𝑙𝑒𝑛(𝑝) + 𝑙𝑒𝑛(𝑡)).
Друга частина це безпосереднє стиснення, воно має один цикл тому складність O(len(s)). Загальна складність O((𝑙𝑒𝑛(𝑝) + 𝑙𝑒𝑛(𝑡))\*len(s)), складність подібна до O(n^2)

## Порядок виконання роботи

1. Створити Notebook-документ і реалізувати приклади, що розглядаються у цій роботі, та виконати завдання до самостійної роботи.
2. Надати відповіді на контрольні питання.
3. Робочий документ оформити у вигляді Notebook-документа (файл .ipynb).
4. Скомпілювати звіт у форматі .html. Для цього необхідно завантажити термінал у робочому каталозі й у командному рядку запустити таку команду:

      `jupyter nbconvert lab_8_StudentLastName.ipynb --to html`

   або, якщо наявна робота з налагодженим середовищем Quarto:

      `quarto preview lab_8_StudentLastName.ipynb`

Проєкт має складатися мінімум з двох файлів: `lab_9_StudentLastName.ipynb` та `lab_9_StudentLastName.5tml`. 

5. Дати відповіді на контрольні запитання.

## Контрольні запитання

1. Що таке «префікс-функція» у контексті алгоритмів на рядках, і як вона відрізняється від Z-функції?

Z-функція – це масив довжини 𝑛, 𝑖-ий елемент якого дорівнює найбільшому числу символів, починаючи з позиції 𝑖, які співпадають із першими символами рядка 𝑠.

Префікс-функція — це масив чисел, де кожне число показує довжину найдовшого підрядка, що є і префіксом і суфіксом.


2. Що таке Z-функція у вищому контексті алгоритмів на рядках, і яке вона має значення для розв’язання задач?

Z-функція – це масив довжини 𝑛, 𝑖-ий елемент якого дорівнює найбільшому числу символів, починаючи з позиції 𝑖, які співпадають із першими символами рядка 𝑠.
Вона використовується для обчислень у алгоритмах, наприклад у пошуку підрядка за відповідним значенням $z[i + len(p) + 1]$ можна зрозуміти, чи входить зразок у текст. Також z-функція використовується для стиснення рядка та пошуку кількості різних підрядків у рядку.

3. Які існують підходи до розв’язання задачі «найдовший спільний підрядок» для двох рядків?

Основні алгоритми пошуку: наївний пошук, алгоритм Рабіна-Карпа, Алгоритм Кнута-Моріса-Прата, Алгоритм Бойера-Мура-Хорспула, пошук за допомогою скінчних автоматів. У наївному пошуку йде просте порівняння двох рядків, а в інших алгоритмах йде передопрацювання, що дає можливість покращити час виконання.

4. Як можна застосувати алгоритми на рядках у задачах обробки природної мови або обробки текстів?

Вони можуть застосовуватись, наприклад, в текстових редакторах або пошукових системах, СУБД, де важливо порівнювати текст та знаходити співпадіння.